# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Main

The following code just uses 5000 min for backtesting although data for complete year is present to change backtesting timeperiod just change the df.head(X)

In [ ]:
df = pd.read_csv('/content/make_market.csv')
df['close']=df['Close']
df['high']=df['High']
df['low']=df['Low']
df['open']=df['Open']
df['volume']=df['Volume']
df.drop(['signal','signals','bid','ask'], axis=1, inplace=True)
df=df.head(5000)
df

,datetime,Open,High,Low,Close,Volume,MACD,Signal_Line,close,high,low,open,volume
0,2023-01-01 00:00:00+00:00,16531.83,16532.00,16527.62,16531.00,10.449073,0.000000,0.000000,16531.00,16532.00,16527.62,16531.83,10.449073
1,2023-01-01 00:01:00+00:00,16531.00,16532.69,16527.14,16529.38,5.961638,-0.129231,-0.025846,16529.38,16532.69,16527.14,16531.00,5.961638
2,2023-01-01 00:02:00+00:00,16528.94,16531.04,16525.01,16527.07,23.837786,-0.413281,-0.103333,16527.07,16531.04,16525.01,16528.94,23.837786
3,2023-01-01 00:03:00+00:00,16525.82,16527.07,16520.14,16522.65,15.802923,-0.983710,-0.279408,16522.65,16527.07,16520.14,16525.82,15.802923
4,2023-01-01 00:04:00+00:00,16521.81,16524.33,16516.41,16523.48,18.666408,-1.353206,-0.494168,16523.48,16524.33,16516.41,16521.81,18.666408
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-01-04 11:15:00+00:00,16846.14,16849.17,16845.84,16847.40,14.655463,2.096476,1.213740,16847.40,16849.17,16845.84,16846.14,14.655463
4996,2023-01-04 11:16:00+00:00,16847.44,16849.71,16843.75,16845.17,7.646157,2.188133,1.408618,16845.17,16849.71,16843.75,16847.44,7.646157
4997,2023-01-04 11:17:00+00:00,16845.05,16846.31,16843.69,16844.08,5.535503,2.148057,1.556506,16844.08,16846.31,16843.69,16845.05,5.535503
4998,2023-01-04 11:18:00+00:00,16843.73,16846.18,16842.57,16845.40,6.214145,2.197478,1.684700,16845.40,16846.18,16842.57,16843.73,6.214145


# Functions

In [ ]:
from collections import deque

inventory_queue=deque()
def square_off(postion):
  if inventory_queue:
    return inventory_queue.pop()
  return None

def add_position(position):
    inventory_queue.append(position)

def time_square_off(current_price):
    if inventory_queue:
        max_profit_position=None
        max_profit=-float('inf')
        profit_made=0
        for i,position in enumerate(inventory_queue):
            if position['type']=='long':
                profit=current_price-position['price']
                if profit>max_profit:
                    max_profit=profit
                    max_profit_position=i
        if max_profit_position is not None:
            squared_off_position=inventory_queue.pop(max_profit_position)
            profit_made=max_profit*squared_off_position['quantity']
            return list(inventory_queue),profit_made
    return list(inventory_queue),0

def view_inventory():
    return list(inventory_queue)


In [ ]:
position=len(inventory_queue)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

class FindingRMSVol:
    def _init_(self, price_data, window=120):
        self.price_data = price_data
        self.window = window

    def realized_vol_1(self):
        log_returns = np.log(self.price_data['Close'] / self.price_data['Close'].shift(1))
        return log_returns.rolling(self.window).std() * np.sqrt(252)

    def realized_vol_2(self):
        log_high_low = np.log(self.price_data['High'] / self.price_data['Low']) ** 2
        log_close_open = np.log(self.price_data['Close'] / self.price_data['Open']) ** 2
        return (0.5 * log_high_low - (2 * np.log(2) - 1) * log_close_open).rolling(self.window).sum() * np.sqrt(252)

    def realized_vol_3(self):
        term1 = np.log(self.price_data['High'] / self.price_data['Open']) * np.log(self.price_data['High'] / self.price_data['Close'])
        term2 = np.log(self.price_data['Low'] / self.price_data['Open']) * np.log(self.price_data['Low'] / self.price_data['Close'])
        return (term1 + term2).rolling(self.window).sum() * np.sqrt(252)

    def realized_vol_4(self):
        return np.log(self.price_data['High'] / self.price_data['Low']).rolling(self.window).std() * np.sqrt(252)

    def min_max_scale_window(self, series, window_size, min_val=0, max_val=1):
        def scale_window(window):
            if len(window) == 0:
                return np.nan
            scaler = MinMaxScaler(feature_range=(min_val, max_val))
            return scaler.fit_transform(window.values.reshape(-1, 1)).flatten()

        return series.rolling(window=window_size).apply(scale_window, raw=False)

    def compute_rms_vol(self):
        # Calculate realized volatilities
        self.price_data['realized_vol_1'] = self.realized_vol_1()
        self.price_data['realized_vol_2'] = self.realized_vol_2()
        self.price_data['realized_vol_3'] = self.realized_vol_3()
        self.price_data['realized_vol_4'] = self.realized_vol_4()

        # Initialize scaled volatility columns
        self.price_data['scaled_vol_1'] = np.nan
        self.price_data['scaled_vol_2'] = np.nan
        self.price_data['scaled_vol_3'] = np.nan
        self.price_data['scaled_vol_4'] = np.nan

        # Scale the last 15 rows for each volatility measure
        for col in ['realized_vol_1', 'realized_vol_2', 'realized_vol_3', 'realized_vol_4']:
            last_15_rows = self.price_data[col].iloc[-15:]
            scaled_last_15 = self.min_max_scale_window(last_15_rows, window_size=len(last_15_rows))
            self.price_data['scaled_' + col] = np.nan
            self.price_data['scaled_' + col].iloc[-15:] = scaled_last_15

        # Calculate RMS volatility
        self.price_data['rms_vol'] = np.sqrt(
            (self.price_data['scaled_realized_vol_1'] ** 2 +
             self.price_data['scaled_realized_vol_2'] ** 2 +
             self.price_data['scaled_realized_vol_3'] ** 2 +
             self.price_data['scaled_realized_vol_4'] ** 2) / 4
        )

        # Drop intermediate columns
        self.price_data.drop(['scaled_realized_vol_1', 'scaled_realized_vol_2',
                              'scaled_realized_vol_3', 'scaled_realized_vol_4',
                              'realized_vol_1', 'realized_vol_2', 'realized_vol_3', 'realized_vol_4'],
                             axis=1, inplace=True)

    def get_data(self):
        return self.price_data

In [ ]:
class MM_calc:
    def __init__(self, df):
        self.df = df

    def calc_fv(self):
        self.df['time_weighted_avg'] = 0
        self.df['volume_weighted_avg'] = 0
        self.df['time_weighted_avg'] = self.df['Close'].ewm(span=15, adjust=False).mean()
        self.df['volume_weighted_avg'] = self.df['time_weighted_avg'].rolling(window=15).apply(
            lambda x: np.average(x, weights=self.df['volume'][x.index].rolling(window=15).apply(np.sum)) # Use the same index as x to select the correct weights
        )
        self.df['fv'] = self.df['volume_weighted_avg'].rolling(window=15).apply(
            lambda x: np.sqrt(np.mean(x**2))
        )
        self.df.drop(['time_weighted_avg', 'volume_weighted_avg'], axis=1, inplace=True)
        return self.df

    def calculate_spread(self):
        spread_values = []

        for i in range(len(self.df)):
            if i < 14:
                spread_values.append(None)
            else:
                last_15_rows = self.df.iloc[i-14:i+1]
                volume_weighted_spread = (last_15_rows['High'] - last_15_rows['Low']) * last_15_rows['Volume']
                total_volume = last_15_rows['Volume'].sum()
                volume_weighted_spread_sum = volume_weighted_spread.sum() / total_volume

                exp_weights = np.exp(1.5 - np.arange(0, 15) * 0.1) * last_15_rows['Volume']
                weighted_mean_spread = (volume_weighted_spread * exp_weights).sum() / exp_weights.sum()

                spread_values.append(weighted_mean_spread)

        self.df['spread'] = spread_values
        return self.df

    # def calc_skew(self):
    #     self.df['skew'] = 0
    #     for i in range(len(self.df)):
    #         position = self.df.iloc[i]['position']
    #         risk_factor = self.df.iloc[i]['rms_vol']
    #         skew = -position * 0.03 * (1 + risk_factor)
    #         # 0.03 sudhar lio agar dhang ka nhi hai to
    #         self.df.at[i, 'skew'] = skew
    #     return self.df

# import numpy as np
# import pandas as pd

# class MM_calc:
#     def __init__(self, df):
#         self.df = df

#     def calc_fv(self):
#         self.df['time_weighted_avg'] = self.df['close'].ewm(span=15, adjust=False).mean()

#         def weighted_average(window):
#             return np.average(window, weights=self.df.loc[window.index, 'volume'])

#         def root_mean_square(window):
#             return np.sqrt(np.mean(window**2))

#         self.df['volume_weighted_avg'] = self.df['time_weighted_avg'].rolling(window=15).apply(weighted_average, raw=False)
#         self.df['fv'] = self.df['volume_weighted_avg'].rolling(window=15).apply(root_mean_square, raw=False)
#         self.df.drop(['time_weighted_avg', 'volume_weighted_avg'], axis=1, inplace=True)

#         return self.df

#     def calculate_spread(self):
#         spread_values = []

#         for i in range(len(self.df)):
#             if i < 14:
#                 spread_values.append(None)
#             else:
#                 last_15_rows = self.df.iloc[i-14:i+1]
#                 volume_weighted_spread = (last_15_rows['High'] - last_15_rows['Low']) * last_15_rows['Volume']
#                 total_volume = last_15_rows['Volume'].sum()
#                 volume_weighted_spread_sum = volume_weighted_spread.sum() / total_volume

#                 exp_weights = np.exp(1.5 - np.arange(0, 15) * 0.1) * last_15_rows['Volume']
#                 weighted_mean_spread = (volume_weighted_spread * exp_weights).sum() / exp_weights.sum()

#                 spread_values.append(weighted_mean_spread)

#         self.df['spread'] = spread_values
#         return self.df

#     def calc_skew(self):
#         self.df['skew'] = 0
#         for i in range(len(self.df)):
#             position = self.df.iloc[i]['position']
#             risk_factor = self.df.iloc[i]['rms_vol']
#             skew = -position * 0.03 * (1 + risk_factor)
#             self.df.at[i, 'skew'] = skew

#         return self.df

# Implementation

In [ ]:
x=MM_calc(df)
df=x.calc_fv()
df=x.calculate_spread()
df

,datetime,Open,High,Low,Close,Volume,MACD,Signal_Line,close,high,low,open,volume,fv,spread
0,2023-01-01 00:00:00+00:00,16531.83,16532.00,16527.62,16531.00,10.449073,0.000000,0.000000,16531.00,16532.00,16527.62,16531.83,10.449073,NaN,NaN
1,2023-01-01 00:01:00+00:00,16531.00,16532.69,16527.14,16529.38,5.961638,-0.129231,-0.025846,16529.38,16532.69,16527.14,16531.00,5.961638,NaN,NaN
2,2023-01-01 00:02:00+00:00,16528.94,16531.04,16525.01,16527.07,23.837786,-0.413281,-0.103333,16527.07,16531.04,16525.01,16528.94,23.837786,NaN,NaN
3,2023-01-01 00:03:00+00:00,16525.82,16527.07,16520.14,16522.65,15.802923,-0.983710,-0.279408,16522.65,16527.07,16520.14,16525.82,15.802923,NaN,NaN
4,2023-01-01 00:04:00+00:00,16521.81,16524.33,16516.41,16523.48,18.666408,-1.353206,-0.494168,16523.48,16524.33,16516.41,16521.81,18.666408,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-01-04 11:15:00+00:00,16846.14,16849.17,16845.84,16847.40,14.655463,2.096476,1.213740,16847.40,16849.17,16845.84,16846.14,14.655463,NaN,21.131176
4996,2023-01-04 11:16:00+00:00,16847.44,16849.71,16843.75,16845.17,7.646157,2.188133,1.408618,16845.17,16849.71,16843.75,16847.44,7.646157,NaN,23.238032
4997,2023-01-04 11:17:00+00:00,16845.05,16846.31,16843.69,16844.08,5.535503,2.148057,1.556506,16844.08,16846.31,16843.69,16845.05,5.535503,NaN,23.908172
4998,2023-01-04 11:18:00+00:00,16843.73,16846.18,16842.57,16845.40,6.214145,2.197478,1.684700,16845.40,16846.18,16842.57,16843.73,6.214145,NaN,25.346183


In [ ]:
df=MM_calc(df).calc_fv()
df=MM_calc(df).calculate_spread()
# df=MM_calc(df).calc_skew()
df

,datetime,Open,High,Low,Close,Volume,MACD,Signal_Line,close,high,low,open,volume,fv,spread
0,2023-01-01 00:00:00+00:00,16531.83,16532.00,16527.62,16531.00,10.449073,0.000000,0.000000,16531.00,16532.00,16527.62,16531.83,10.449073,NaN,NaN
1,2023-01-01 00:01:00+00:00,16531.00,16532.69,16527.14,16529.38,5.961638,-0.129231,-0.025846,16529.38,16532.69,16527.14,16531.00,5.961638,NaN,NaN
2,2023-01-01 00:02:00+00:00,16528.94,16531.04,16525.01,16527.07,23.837786,-0.413281,-0.103333,16527.07,16531.04,16525.01,16528.94,23.837786,NaN,NaN
3,2023-01-01 00:03:00+00:00,16525.82,16527.07,16520.14,16522.65,15.802923,-0.983710,-0.279408,16522.65,16527.07,16520.14,16525.82,15.802923,NaN,NaN
4,2023-01-01 00:04:00+00:00,16521.81,16524.33,16516.41,16523.48,18.666408,-1.353206,-0.494168,16523.48,16524.33,16516.41,16521.81,18.666408,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-01-04 11:15:00+00:00,16846.14,16849.17,16845.84,16847.40,14.655463,2.096476,1.213740,16847.40,16849.17,16845.84,16846.14,14.655463,NaN,21.131176
4996,2023-01-04 11:16:00+00:00,16847.44,16849.71,16843.75,16845.17,7.646157,2.188133,1.408618,16845.17,16849.71,16843.75,16847.44,7.646157,NaN,23.238032
4997,2023-01-04 11:17:00+00:00,16845.05,16846.31,16843.69,16844.08,5.535503,2.148057,1.556506,16844.08,16846.31,16843.69,16845.05,5.535503,NaN,23.908172
4998,2023-01-04 11:18:00+00:00,16843.73,16846.18,16842.57,16845.40,6.214145,2.197478,1.684700,16845.40,16846.18,16842.57,16843.73,6.214145,NaN,25.346183


In [ ]:
df['time_weighted_avg'] = df['Close'].ewm(span=15, adjust=False).mean()
# nan handle
df['time_weighted_avg'] = df['time_weighted_avg'].fillna(method='ffill')
df['volume_weighted_avg']=0
df['volume_weighted_avg'] = df['time_weighted_avg'].rolling(window=15).apply(
            lambda x: np.average(x, weights=df['volume'][x.index].rolling(window=15).apply(np.sum)) # Use the same index as x to select the correct weights
        )

In [ ]:
df['fv'] = df['volume_weighted_avg'].rolling(window=15).apply(
            lambda x: np.sqrt(np.mean(x**2))
        )

In [ ]:
df['fv']=df['Close'].ewm(span=15, adjust=False).mean()

In [ ]:
df

,datetime,Open,High,Low,Close,Volume,MACD,Signal_Line,close,high,low,open,volume,fv,spread,time_weighted_avg,volume_weighted_avg
0,2023-01-01 00:00:00+00:00,16531.83,16532.00,16527.62,16531.00,10.449073,0.000000,0.000000,16531.00,16532.00,16527.62,16531.83,10.449073,16531.000000,NaN,16531.000000,NaN
1,2023-01-01 00:01:00+00:00,16531.00,16532.69,16527.14,16529.38,5.961638,-0.129231,-0.025846,16529.38,16532.69,16527.14,16531.00,5.961638,16530.797500,NaN,16530.797500,NaN
2,2023-01-01 00:02:00+00:00,16528.94,16531.04,16525.01,16527.07,23.837786,-0.413281,-0.103333,16527.07,16531.04,16525.01,16528.94,23.837786,16530.331562,NaN,16530.331562,NaN
3,2023-01-01 00:03:00+00:00,16525.82,16527.07,16520.14,16522.65,15.802923,-0.983710,-0.279408,16522.65,16527.07,16520.14,16525.82,15.802923,16529.371367,NaN,16529.371367,NaN
4,2023-01-01 00:04:00+00:00,16521.81,16524.33,16516.41,16523.48,18.666408,-1.353206,-0.494168,16523.48,16524.33,16516.41,16521.81,18.666408,16528.634946,NaN,16528.634946,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-01-04 11:15:00+00:00,16846.14,16849.17,16845.84,16847.40,14.655463,2.096476,1.213740,16847.40,16849.17,16845.84,16846.14,14.655463,16841.440802,21.131176,16841.440802,NaN
4996,2023-01-04 11:16:00+00:00,16847.44,16849.71,16843.75,16845.17,7.646157,2.188133,1.408618,16845.17,16849.71,16843.75,16847.44,7.646157,16841.906952,23.238032,16841.906952,NaN
4997,2023-01-04 11:17:00+00:00,16845.05,16846.31,16843.69,16844.08,5.535503,2.148057,1.556506,16844.08,16846.31,16843.69,16845.05,5.535503,16842.178583,23.908172,16842.178583,NaN
4998,2023-01-04 11:18:00+00:00,16843.73,16846.18,16842.57,16845.40,6.214145,2.197478,1.684700,16845.40,16846.18,16842.57,16843.73,6.214145,16842.581260,25.346183,16842.581260,NaN


In [ ]:
def generate_quotes(df):
  df['bid']=df['fv']-(df['spread']/100)
  df['ask']= df['fv'] + (df['spread']/100)
  return df

In [ ]:
df=generate_quotes(df)
df

# df= FindingRMSVol(df)
# df['skew']= 0.03* df['rms_vol']

# def generate_quotes(df):
#   df['bid']=df['fv']-(df['spread']/100)-df['skew']
#   df['ask']= df['fv'] + (df['spread']/100) - df['skew']
#   return df
# df=generate_quotes(df)
# df

,datetime,Open,High,Low,Close,Volume,MACD,Signal_Line,close,high,low,open,volume,fv,spread,time_weighted_avg,volume_weighted_avg,bid,ask
0,2023-01-01 00:00:00+00:00,16531.83,16532.00,16527.62,16531.00,10.449073,0.000000,0.000000,16531.00,16532.00,16527.62,16531.83,10.449073,16531.000000,NaN,16531.000000,NaN,NaN,NaN
1,2023-01-01 00:01:00+00:00,16531.00,16532.69,16527.14,16529.38,5.961638,-0.129231,-0.025846,16529.38,16532.69,16527.14,16531.00,5.961638,16530.797500,NaN,16530.797500,NaN,NaN,NaN
2,2023-01-01 00:02:00+00:00,16528.94,16531.04,16525.01,16527.07,23.837786,-0.413281,-0.103333,16527.07,16531.04,16525.01,16528.94,23.837786,16530.331562,NaN,16530.331562,NaN,NaN,NaN
3,2023-01-01 00:03:00+00:00,16525.82,16527.07,16520.14,16522.65,15.802923,-0.983710,-0.279408,16522.65,16527.07,16520.14,16525.82,15.802923,16529.371367,NaN,16529.371367,NaN,NaN,NaN
4,2023-01-01 00:04:00+00:00,16521.81,16524.33,16516.41,16523.48,18.666408,-1.353206,-0.494168,16523.48,16524.33,16516.41,16521.81,18.666408,16528.634946,NaN,16528.634946,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-01-04 11:15:00+00:00,16846.14,16849.17,16845.84,16847.40,14.655463,2.096476,1.213740,16847.40,16849.17,16845.84,16846.14,14.655463,16841.440802,21.131176,16841.440802,NaN,16841.229490,16841.652114
4996,2023-01-04 11:16:00+00:00,16847.44,16849.71,16843.75,16845.17,7.646157,2.188133,1.408618,16845.17,16849.71,16843.75,16847.44,7.646157,16841.906952,23.238032,16841.906952,NaN,16841.674571,16842.139332
4997,2023-01-04 11:17:00+00:00,16845.05,16846.31,16843.69,16844.08,5.535503,2.148057,1.556506,16844.08,16846.31,16843.69,16845.05,5.535503,16842.178583,23.908172,16842.178583,NaN,16841.939501,16842.417664
4998,2023-01-04 11:18:00+00:00,16843.73,16846.18,16842.57,16845.40,6.214145,2.197478,1.684700,16845.40,16846.18,16842.57,16843.73,6.214145,16842.581260,25.346183,16842.581260,NaN,16842.327798,16842.834722


In [ ]:
# def generate_signals(df):
#   df['signal']=0
#   for i in range(len(df)):
#     if(df.iloc[i]['low']<=df.iloc[i]['bid']):
#       df.iloc[i]['signal']= df.iloc[i]['signal']+1
#       position = {
#       'type': 'long',
#       'price': df.iloc[i]['bid'],
#       'quantity': 1,
#       'sq_price': df.iloc[i]['ask']
#     # quantity kese change kren
#       }
#       if(df.iloc[i]['high']<=df.iloc[i]['ask']):
#         df.iloc[i]['signal']=df.iloc[i]['signal'] -1
#         position = {
#        'type': 'short',
#        'price': df.iloc[i]['ask'],
#        'quantity': 1,
#        'sq_price': df.iloc[i]['bid']
#     # quantity kese change kren
#       }
#   return df

In [ ]:
def generate_signals(df):
  df['signals']=0
  for i in range(len(df)):
    if(df['low'].iloc[i]<=df['bid'].iloc[i]):
      df['signals'].iloc[i]= 1
    elif(df['high'].iloc[i]>=df['ask'].iloc[i]):
      df['signals'].iloc[i]= -1
  return df

In [ ]:
df=generate_signals(df)
df

,datetime,Open,High,Low,Close,Volume,MACD,Signal_Line,close,high,low,open,volume,fv,spread,time_weighted_avg,volume_weighted_avg,bid,ask,signals
0,2023-01-01 00:00:00+00:00,16531.83,16532.00,16527.62,16531.00,10.449073,0.000000,0.000000,16531.00,16532.00,16527.62,16531.83,10.449073,16531.000000,NaN,16531.000000,NaN,NaN,NaN,0
1,2023-01-01 00:01:00+00:00,16531.00,16532.69,16527.14,16529.38,5.961638,-0.129231,-0.025846,16529.38,16532.69,16527.14,16531.00,5.961638,16530.797500,NaN,16530.797500,NaN,NaN,NaN,0
2,2023-01-01 00:02:00+00:00,16528.94,16531.04,16525.01,16527.07,23.837786,-0.413281,-0.103333,16527.07,16531.04,16525.01,16528.94,23.837786,16530.331562,NaN,16530.331562,NaN,NaN,NaN,0
3,2023-01-01 00:03:00+00:00,16525.82,16527.07,16520.14,16522.65,15.802923,-0.983710,-0.279408,16522.65,16527.07,16520.14,16525.82,15.802923,16529.371367,NaN,16529.371367,NaN,NaN,NaN,0
4,2023-01-01 00:04:00+00:00,16521.81,16524.33,16516.41,16523.48,18.666408,-1.353206,-0.494168,16523.48,16524.33,16516.41,16521.81,18.666408,16528.634946,NaN,16528.634946,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-01-04 11:15:00+00:00,16846.14,16849.17,16845.84,16847.40,14.655463,2.096476,1.213740,16847.40,16849.17,16845.84,16846.14,14.655463,16841.440802,21.131176,16841.440802,NaN,16841.229490,16841.652114,-1
4996,2023-01-04 11:16:00+00:00,16847.44,16849.71,16843.75,16845.17,7.646157,2.188133,1.408618,16845.17,16849.71,16843.75,16847.44,7.646157,16841.906952,23.238032,16841.906952,NaN,16841.674571,16842.139332,-1
4997,2023-01-04 11:17:00+00:00,16845.05,16846.31,16843.69,16844.08,5.535503,2.148057,1.556506,16844.08,16846.31,16843.69,16845.05,5.535503,16842.178583,23.908172,16842.178583,NaN,16841.939501,16842.417664,-1
4998,2023-01-04 11:18:00+00:00,16843.73,16846.18,16842.57,16845.40,6.214145,2.197478,1.684700,16845.40,16846.18,16842.57,16843.73,6.214145,16842.581260,25.346183,16842.581260,NaN,16842.327798,16842.834722,-1


# Backtesting Engine (Multi-trade)

In [ ]:
def backtest(data_bt):

  balance=1000000
  capital=1000000
  capital_per_trade_percent=10
  capital_per_trade=(capital_per_trade_percent/100)*capital

  buys=[]
  sells=[]
  compounding_capital=[]
  drawdown=[]

  positions=0
  max_positions=10
  j=0

  data_bt['TR'] = np.maximum(data_bt['High'] - data_bt['Low'],
                            np.maximum(abs(data_bt['High'] - data_bt['Close'].shift(1)),
                                       abs(data_bt['Low'] - data_bt['Close'].shift(1))))
  data_bt['ATR'] = data_bt['TR'].rolling(window=14).mean()

  leverage=5

  normal_stop_loss=0.1
  normal_take_profit=5
  ATR_multiplier=2
  Intraday_price_change_limit=500


  twd1=pd.DataFrame(columns=['Entry Index','Exit Index','Open/Close','Trade Duration','Entry Price','Exit Price','sop','stop loss','take profit','ATR_stop_loss','Returns for Trade in %','Type of Trade','Trade Close By','No. of Stocks Traded','Profit/Loss','Capital'])
  twd2=pd.DataFrame(columns=['Entry Index','Exit Index','Open/Close','Trade Duration','Entry Price','Exit Price','sop','stop loss','take profit','ATR_stop_loss','Returns for Trade in %','Type of Trade','Trade Close By','No. of Stocks Traded','Profit/Loss','Capital'])
  twd3=pd.DataFrame(columns=['Entry Index','Exit Index','Open/Close','Trade Duration','Entry Price','Exit Price','sop','stop loss','take profit','ATR_stop_loss','Returns for Trade in %','Type of Trade','Trade Close By','No. of Stocks Traded','Profit/Loss','Capital'])

  data_bt['TR'] = np.maximum(data_bt['High'] - data_bt['Low'],
                            np.maximum(abs(data_bt['High'] - data_bt['Close'].shift(1)),
                                       abs(data_bt['Low'] - data_bt['Close'].shift(1))))
  data_bt['ATR'] = data_bt['TR'].rolling(window=14).mean()


  for i in range(len(data_bt)):
    if(capital>0):

      if(data_bt['ATR'].iloc[i]>=2.5):
        leverage=3
        multiplier3=1.5
        multiplier3=1.5
      elif(data_bt['ATR'].iloc[i]>=2 and data_bt['ATR'].iloc[i]<=2.5):
        leverage=4
        multiplier3=2
        multiplier3=2
      else:
        leverage=5
        multiplie3=2
        multiplier3=2



      for k in range(0,len(twd1)):
        if(twd1['Open/Close'].iloc[k]==1):

          twd1['ATR_stop_loss'].iloc[k]=twd1['Entry Price'].iloc[k]-(ATR_multiplier*data_bt['ATR'].iloc[i])

          if(twd1['sop'].iloc[k]<=data_bt['High'].iloc[i]):
            capital=capital+(twd1['No. of Stocks Traded'].iloc[k]*(data_bt['ask'].iloc[i]-twd1['Entry Price'].iloc[k]))
            returns=(((data_bt['ask'].iloc[i]-twd1['Entry Price'].iloc[k])/twd1['Entry Price'].iloc[k])*100)
            net=twd1['No. of Stocks Traded'].iloc[k]*((data_bt['ask'].iloc[i]-twd1['Entry Price'].iloc[k]))
            twd1['Exit Index'].iloc[k]=i
            twd1['Exit Price'].iloc[k]=data_bt['ask'].iloc[i]
            twd1['Open/Close'].iloc[k]=0
            twd1['Trade Duration'].iloc[k]=twd1['Exit Index'].iloc[k]-twd1['Entry Index'].iloc[k]
            twd1['Returns for Trade in %'].iloc[k]=returns
            twd1['Profit/Loss'].iloc[k]=net
            twd1['Capital'].iloc[k]=capital
            twd1['Trade Close By'].iloc[k]="make_market"
            new_row = {'Entry Index':twd1['Entry Index'].iloc[k],'Exit Index':i,'Trade Duration':twd1['Exit Index'].iloc[k]-twd1['Entry Index'].iloc[k],'Entry Price':twd1['Entry Price'].iloc[k],'Exit Price':twd1['Exit Price'].iloc[k],'Returns for Trade in %':returns,'Type of Trade':"long",
                     'Trade Close By':"make_market", 'Max Drawdown for Trade':0, 'Max Dip for Trade':0, 'No. of Stocks Traded':twd1['No. of Stocks Traded'].iloc[k],'Profit/Loss':net,'Capital':capital}
            twd2.loc[len(twd2)]=new_row
            # data_bt['signals'].iloc[i]=0
            sells.append(i)
            positions-=1

          elif(data_bt['ask'].iloc[i]<=(1-normal_stop_loss/100)*(twd1['Entry Price'].iloc[k])):
            capital=capital+(twd1['No. of Stocks Traded'].iloc[k]*(data_bt['ask'].iloc[i]-twd1['Entry Price'].iloc[k]))
            returns=(((data_bt['ask'].iloc[i]-twd1['Entry Price'].iloc[k])/twd1['Entry Price'].iloc[k])*100)
            net=twd1['No. of Stocks Traded'].iloc[k]*((data_bt['ask'].iloc[i]-twd1['Entry Price'].iloc[k]))
            twd1['Exit Index'].iloc[k]=i
            twd1['Exit Price'].iloc[k]=data_bt['ask'].iloc[i]
            twd1['Open/Close'].iloc[k]=0
            twd1['Trade Duration'].iloc[k]=twd1['Exit Index'].iloc[k]-twd1['Entry Index'].iloc[k]
            twd1['Returns for Trade in %'].iloc[k]=returns
            twd1['Profit/Loss'].iloc[k]=net
            twd1['Capital'].iloc[k]=capital
            twd1['Trade Close By'].iloc[k]="stop_loss"
            new_row = {'Entry Index':twd1['Entry Index'].iloc[k],'Exit Index':i,'Trade Duration':twd1['Exit Index'].iloc[k]-twd1['Entry Index'].iloc[k],'Entry Price':twd1['Entry Price'].iloc[k],'Exit Price':twd1['Exit Price'].iloc[k],'Returns for Trade in %':returns,'Type of Trade':"long",
                     'Trade Close By':"stop_loss", 'Max Drawdown for Trade':0, 'Max Dip for Trade':0, 'No. of Stocks Traded':twd1['No. of Stocks Traded'].iloc[k],'Profit/Loss':net,'Capital':capital}
            twd3.loc[len(twd3)]=new_row
            # data_bt['signals'].iloc[i]=0
            sells.append(i)
            positions-=1



        elif(twd1['Open/Close'].iloc[k]==-1):

          twd1['ATR_stop_loss'].iloc[k]=twd1['Entry Price'].iloc[k]+(ATR_multiplier*data_bt['ATR'].iloc[i])

          if(data_bt['Low'].iloc[i]<=twd1['sop'].iloc[k]):
            capital=capital+(twd1['No. of Stocks Traded'].iloc[k]*(twd1['Entry Price'].iloc[k]-data_bt['bid'].iloc[i]))
            returns=(((twd1['Entry Price'].iloc[k]-data_bt['bid'].iloc[i])/twd1['Entry Price'].iloc[k])*100)
            net=twd1['No. of Stocks Traded'].iloc[k]*(twd1['Entry Price'].iloc[k]-data_bt['bid'].iloc[i])
            twd1['Exit Index'].iloc[k]=i
            twd1['Exit Price'].iloc[k]=data_bt['bid'].iloc[i]
            twd1['Open/Close'].iloc[k]=0
            twd1['Trade Duration'].iloc[k]=twd1['Exit Index'].iloc[k]-twd1['Entry Index'].iloc[k]
            twd1['Returns for Trade in %'].iloc[k]=returns
            twd1['Profit/Loss'].iloc[k]=net
            twd1['Capital'].iloc[k]=capital
            twd1['Trade Close By'].iloc[k]="make_market"
            new_row = {'Entry Index':twd1['Entry Index'].iloc[k],'Exit Index':i,'Trade Duration':twd1['Exit Index'].iloc[k]-twd1['Entry Index'].iloc[k],'Entry Price':twd1['Entry Price'].iloc[k],'Exit Price':twd1['Exit Price'].iloc[k],'Returns for Trade in %':returns,'Type of Trade':"short",
                     'Trade Close By':"make_market", 'Max Drawdown for Trade':0, 'Max Dip for Trade':0, 'No. of Stocks Traded':twd1['No. of Stocks Traded'].iloc[k],'Profit/Loss':net,'Capital':capital}
            twd2.loc[len(twd2)]=new_row
            # data_bt['signals'].iloc[i]=0
            buys.append(i)
            positions-=1

          elif(data_bt['bid'].iloc[i]>=(1+normal_stop_loss/100)*(twd1['Entry Price'].iloc[k])):
            capital=capital+(twd1['No. of Stocks Traded'].iloc[k]*(twd1['Entry Price'].iloc[k]-data_bt['bid'].iloc[i]))
            returns=(((twd1['Entry Price'].iloc[k]-data_bt['bid'].iloc[i])/twd1['Entry Price'].iloc[k])*100)
            net=twd1['No. of Stocks Traded'].iloc[k]*((twd1['Entry Price'].iloc[k]-data_bt['bid'].iloc[k]))
            twd1['Exit Index'].iloc[k]=i
            twd1['Exit Price'].iloc[k]=data_bt['bid'].iloc[i]
            twd1['Open/Close'].iloc[k]=0
            twd1['Trade Duration'].iloc[k]=twd1['Exit Index'].iloc[k]-twd1['Entry Index'].iloc[k]
            twd1['Returns for Trade in %'].iloc[k]=returns
            twd1['Profit/Loss'].iloc[k]=net
            twd1['Capital'].iloc[k]=capital
            twd1['Trade Close By'].iloc[k]="stop_loss"
            new_row = {'Entry Index':twd1['Entry Index'].iloc[k],'Exit Index':i,'Trade Duration':twd1['Exit Index'].iloc[k]-twd1['Entry Index'].iloc[k],'Entry Price':twd1['Entry Price'].iloc[k],'Exit Price':twd1['Exit Price'].iloc[k],'Returns for Trade in %':returns,'Type of Trade':"short",
                     'Trade Close By':"stop_loss", 'Max Drawdown for Trade':0, 'Max Dip for Trade':0, 'No. of Stocks Traded':twd1['No. of Stocks Traded'].iloc[k],'Profit/Loss':net,'Capital':capital}
            twd3.loc[len(twd3)]=new_row
            # data_bt['signals'].iloc[i]=0
            sells.append(i)
            positions-=1




      if(data_bt['signals'].iloc[i]==1 and positions<=max_positions):

        positions+=1
        price=data_bt['bid'].iloc[i]
        normal_stop_loss_price=(1-(normal_stop_loss/100))*price
        # normal_take_profit_price=(1+(normal_take_profit/100))*price
        capital_per_trade=(capital_per_trade_percent/100)*capital
        no_of_stocks=(int(capital_per_trade/data_bt['bid'].iloc[i]))*leverage
        buys.append(i)
        new_row = {'Entry Index':i,'Exit Index':0,'Open/Close':1,'Trade Duration':0,'Entry Price':price,'sop':data_bt['ask'].iloc[i],'stop loss':normal_stop_loss_price,'Returns for Trade in %':0,'Type of Trade':"long",
                     'Trade Close By':"none", 'Max Drawdown for Trade':0, 'Max Dip for Trade':0, 'No. of Stocks Traded':no_of_stocks,'Profit/Loss':0,'Capital':0}
        twd1.loc[len(twd1)]=new_row




      elif(data_bt['signals'].iloc[i]==-1 and positions<=max_positions):

        positions+=1
        price=data_bt['ask'].iloc[i]
        normal_stop_loss_price=(1+(normal_stop_loss/100))*price
        # normal_take_profit_price=(1-(normal_take_profit/100))*price
        capital_per_trade=(capital_per_trade_percent/100)*capital
        no_of_stocks=(int(capital_per_trade/data_bt['ask'].iloc[i]))*leverage
        sells.append(i)
        new_row = {'Entry Index':i,'Exit Index':0,'Open/Close':-1,'Trade Duration':0,'Entry Price':price,'sop':data_bt['bid'].iloc[i],'stop loss':normal_stop_loss_price,'Returns for Trade in %':0,'Type of Trade':"short",
                     'Trade Close By':"none", 'Max Drawdown for Trade':0, 'Max Dip for Trade':0, 'No. of Stocks Traded':no_of_stocks,'Profit/Loss':0,'Capital':0}
        twd1.loc[len(twd1)]=new_row




      compounding_capital.append(capital)
      max_capital=max(compounding_capital)
      drawdown.append(((max_capital-capital)/capital)*100)



    else:
      print('No Balance Available')
      break



  for p in range(0,len(twd1)):
    if(twd1['Open/Close'].iloc[p]==1):
            capital=capital+(twd1['No. of Stocks Traded'].iloc[p]*(data_bt['Close'].iloc[i]-twd1['Entry Price'].iloc[p]))
            returns=(((data_bt['Close'].iloc[i]-twd1['Entry Price'].iloc[p])/twd1['Entry Price'].iloc[p])*100)
            net=twd1['No. of Stocks Traded'].iloc[p]*((data_bt['Close'].iloc[i]-twd1['Entry Price'].iloc[p]))
            twd1['Exit Index'].iloc[p]=i
            twd1['Exit Price'].iloc[p]=data_bt['Close'].iloc[i]
            twd1['Open/Close'].iloc[p]=0
            twd1['Trade Duration'].iloc[p]=twd1['Exit Index'].iloc[p]-twd1['Entry Index'].iloc[p]
            twd1['Returns for Trade in %'].iloc[p]=returns
            twd1['Profit/Loss'].iloc[p]=net
            twd1['Capital'].iloc[p]=capital
            twd1['Trade Close By'].iloc[p]="end"
            data_bt['signals'].iloc[i]=0
            sells.append(i)
            positions-=1




    if(twd1['Open/Close'].iloc[p]==-1):
            capital=capital+(twd1['No. of Stocks Traded'].iloc[p]*(twd1['Entry Price'].iloc[p]-data_bt['Close'].iloc[i]))
            returns=(((twd1['Entry Price'].iloc[p]-data_bt['Close'].iloc[i])/twd1['Entry Price'].iloc[p])*100)
            net=twd1['No. of Stocks Traded'].iloc[p]*(twd1['Entry Price'].iloc[p]-data_bt['Close'].iloc[i])
            twd1['Exit Index'].iloc[p]=i
            twd1['Exit Price'].iloc[p]=data_bt['Close'].iloc[i]
            twd1['Open/Close'].iloc[p]=0
            twd1['Trade Duration'].iloc[p]=twd1['Exit Index'].iloc[p]-twd1['Entry Index'].iloc[p]
            twd1['Returns for Trade in %'].iloc[p]=returns
            twd1['Profit/Loss'].iloc[p]=net
            twd1['Capital'].iloc[p]=capital
            twd1['Trade Close By'].iloc[p]="end"
            data_bt['signals'].iloc[i]=0
            buys.append(i)
            positions-=1






  #Code

  #Benchmark Return and Total Returns
  num=(int((balance/data_bt['Close'].iloc[0])))
  br=((num*(data_bt['Close'].iloc[len(data_bt)-1]-data_bt['Close'].iloc[0]))/balance)*100
  num3=(int(balance/data_bt['Close'].iloc[0]))*leverage
  # num4=(int(num3/100))*100
  br2=((num3*(data_bt['Close'].iloc[len(data_bt)-1]-data_bt['Close'].iloc[0]))/balance)*100
  print("Benchmark Return (with leverage):",br2,"%")

  #Trades
  print("Number of Close Trades are:",int(len(twd1)))
  wt=0
  lt=0
  total_returns=0
  for i in range(len(twd1)):
    if(twd1['Returns for Trade in %'].iloc[i]>0):
      wt+=1
    elif(twd1['Returns for Trade in %'].iloc[i]<0):
      lt+=1
    total_returns+=twd1['Returns for Trade in %'].iloc[i]
  print("Number of Winning Trades are:",wt)
  print("Number of Losing Trades are:",lt)
  print("Total Returns of Strategy:",((twd1['Capital'].iloc[len(twd1)-1]-balance)/balance)*100,"%")

  #Gross Profit and Net Profit
  gp=(twd1['Capital'].iloc[len(twd1)-1])-balance
  print("Gross Profit (in %) is:",(gp/balance)*100,"%")
  print("Gross Profit is:",gp)

  #Max Drawdown
  # d=twd1['Max Drawdown for Trade']
  maxDrawdown=max(drawdown)
  print("Maximum Drawdown is:",maxDrawdown,"%")
  #Avg. Drawdown
  avgDrawdown = np.mean(drawdown)
  print("Average Drawdown is:",avgDrawdown,"%")

  #Sharpe Ratio
  # print("Sharpe Ratio of the strategy is:", sharpe_ratio((returns1)))

  return twd1,twd2,twd3

In [ ]:
a1,a2,a3=backtest(df)

Benchmark Return (with leverage): 5.578919999999976 %
Number of Close Trades are: 2951
Number of Winning Trades are: 1876
Number of Losing Trades are: 1075
Total Returns of Strategy: 5.6351065729961265 %
Gross Profit (in %) is: 5.6351065729961265 %
Gross Profit is: 56351.065729961265
Maximum Drawdown is: 1.2863090198385896 %
Average Drawdown is: 0.34705312692482276 %


In [ ]:
a1

,Entry Index,Exit Index,Open/Close,Trade Duration,Entry Price,Exit Price,sop,stop loss,take profit,ATR_stop_loss,Returns for Trade in %,Type of Trade,Trade Close By,No. of Stocks Traded,Profit/Loss,Capital
0,14,18,0,4,16517.786092,16518.611951,16519.962791,16501.268305,NaN,16507.210377,0.005000,long,make_market,18,14.865477,1.000015e+06
1,15,18,0,3,16516.629200,16518.611951,16519.066072,16500.112571,NaN,16506.053486,0.012005,long,make_market,18,35.689522,1.000051e+06
2,16,18,0,2,16515.698123,16518.611951,16518.307740,16499.182424,NaN,16505.122408,0.017643,long,make_market,18,52.448920,1.000103e+06
3,17,18,0,1,16515.366635,16518.611951,16517.940995,16498.851268,NaN,16504.790921,0.019650,long,make_market,18,58.415699,1.000161e+06
4,18,19,0,1,16515.892225,16519.698845,16518.611951,16499.376333,NaN,16504.293653,0.023048,long,make_market,18,68.519164,1.000230e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2946,4990,4991,0,1,16837.867071,16838.511987,16838.208633,16821.029204,NaN,16831.618499,0.003830,long,make_market,18,11.608483,1.056339e+06
2947,4991,4992,0,1,16838.159255,16839.214718,16838.511987,16821.321095,NaN,16831.876397,0.006268,long,make_market,18,18.998347,1.056358e+06
2948,4992,4999,0,7,16839.214718,16840.940000,16838.847618,16856.053933,NaN,16846.794718,-0.010246,short,end,18,-31.055071,1.056372e+06
2949,4995,4999,0,4,16841.652114,16842.093525,16841.229490,16858.493766,NaN,16849.232114,-0.002621,short,make_market,18,-7.945410,1.056368e+06


In [ ]:
a2

,Entry Index,Exit Index,Open/Close,Trade Duration,Entry Price,Exit Price,sop,stop loss,take profit,ATR_stop_loss,Returns for Trade in %,Type of Trade,Trade Close By,No. of Stocks Traded,Profit/Loss,Capital
0,14,18,NaN,4,16517.786092,16518.611951,NaN,NaN,NaN,NaN,0.005000,long,make_market,18,14.865477,1.000015e+06
1,15,18,NaN,3,16516.629200,16518.611951,NaN,NaN,NaN,NaN,0.012005,long,make_market,18,35.689522,1.000051e+06
2,16,18,NaN,2,16515.698123,16518.611951,NaN,NaN,NaN,NaN,0.017643,long,make_market,18,52.448920,1.000103e+06
3,17,18,NaN,1,16515.366635,16518.611951,NaN,NaN,NaN,NaN,0.019650,long,make_market,18,58.415699,1.000161e+06
4,18,19,NaN,1,16515.892225,16519.698845,NaN,NaN,NaN,NaN,0.023048,long,make_market,18,68.519164,1.000230e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503,4989,4990,NaN,1,16837.912522,16838.208633,NaN,NaN,NaN,NaN,0.001759,long,make_market,18,5.330007,1.056327e+06
2504,4990,4991,NaN,1,16837.867071,16838.511987,NaN,NaN,NaN,NaN,0.003830,long,make_market,18,11.608483,1.056339e+06
2505,4991,4992,NaN,1,16838.159255,16839.214718,NaN,NaN,NaN,NaN,0.006268,long,make_market,18,18.998347,1.056358e+06
2506,4914,4995,NaN,81,16840.623404,16841.652114,NaN,NaN,NaN,NaN,0.006109,long,make_market,18,18.516781,1.056376e+06


In [ ]:
a3

,Entry Index,Exit Index,Open/Close,Trade Duration,Entry Price,Exit Price,sop,stop loss,take profit,ATR_stop_loss,Returns for Trade in %,Type of Trade,Trade Close By,No. of Stocks Traded,Profit/Loss,Capital
0,47,116,NaN,69,16517.575320,16534.516051,NaN,NaN,NaN,NaN,0.102562,short,stop_loss,18,65.347430,1.003226e+06
1,51,116,NaN,65,16517.154985,16534.516051,NaN,NaN,NaN,NaN,0.105109,short,stop_loss,18,129.435394,1.003539e+06
2,48,117,NaN,69,16518.823199,16535.380064,NaN,NaN,NaN,NaN,0.100230,short,stop_loss,18,51.013048,1.003837e+06
3,49,118,NaN,69,16519.876827,16536.437733,NaN,NaN,NaN,NaN,0.100248,short,stop_loss,18,48.193031,1.004135e+06
4,50,119,NaN,69,16520.476397,16537.110622,NaN,NaN,NaN,NaN,0.100689,short,stop_loss,18,50.204272,1.004434e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,4824,4914,NaN,90,16865.536083,16847.937228,NaN,NaN,NaN,NaN,-0.104348,long,stop_loss,18,-316.779387,1.057214e+06
428,4825,4925,NaN,100,16861.507912,16841.561998,NaN,NaN,NaN,NaN,-0.118293,long,stop_loss,18,-359.026459,1.056855e+06
429,4826,4926,NaN,100,16858.197955,16841.190440,NaN,NaN,NaN,NaN,-0.100886,long,stop_loss,18,-306.135273,1.056548e+06
430,4868,4927,NaN,59,16857.120258,16838.784517,NaN,NaN,NaN,NaN,-0.108771,long,stop_loss,18,-330.043321,1.056218e+06


In [ ]:
class FindingRMSVol:
    def __init__(self, price_data, window=120):
        self.price_data=price_data
        self.window=window

    def realized_vol_1(self):
        log_returns=np.log(self.price_data['Close']/self.price_data['Close'].shift(1))
        return log_returns.rolling(self.window).std()*np.sqrt(252)

    def realized_vol_2(self):
        log_high_low=np.log(self.price_data['High']/self.price_data['Low'])**2
        log_close_open=np.log(self.price_data['Close']/self.price_data['Open'])**2
        return (0.5*log_high_low-(2*np.log(2)-1)*log_close_open).rolling(self.window).sum()*np.sqrt(252)

    def realized_vol_3(self):
        term1=np.log(self.price_data['High']/self.price_data['Open'])*np.log(self.price_data['High']/self.price_data['Close'])
        term2=np.log(self.price_data['Low']/self.price_data['Open'])*np.log(self.price_data['Low']/self.price_data['Close'])
        return (term1+term2).rolling(self.window).sum()*np.sqrt(252)

    def realized_vol_4(self):
        return np.log(self.price_data['High']/self.price_data['Low']).rolling(self.window).std()*np.sqrt(252)

    def min_max_scale(self, series, min_val=0, max_val=1):
        scaler=MinMaxScaler(feature_range=(min_val,max_val))
        return scaler.fit_transform(series.values.reshape(-1,1)).flatten()

    def compute_rms_vol(self):
        self.price_data['realized_vol_1']=self.realized_vol_1()
        self.price_data['realized_vol_2']=self.realized_vol_2()
        self.price_data['realized_vol_3']=self.realized_vol_3()
        self.price_data['realized_vol_4']=self.realized_vol_4()
        self.price_data['scaled_vol_1']=self.min_max_scale(self.price_data['realized_vol_1'])
        self.price_data['scaled_vol_2']=self.min_max_scale(self.price_data['realized_vol_2'])
        self.price_data['scaled_vol_3']=self.min_max_scale(self.price_data['realized_vol_3'])
        self.price_data['scaled_vol_4']=self.min_max_scale(self.price_data['realized_vol_4'])
        self.price_data['rms_vol']=np.sqrt(
            (self.price_data['scaled_vol_1']**2+
             self.price_data['scaled_vol_2']**2+
             self.price_data['scaled_vol_3']**2+
             self.price_data['scaled_vol_4']**2)/4
        )
        self.price_data.drop(['scaled_vol_1','scaled_vol_2','scaled_vol_3','scaled_vol_4',
                              'realized_vol_1','realized_vol_2','realized_vol_3','realized_vol_4'],
                             axis=1,inplace=True)

    def get_data(self):
        return self.price_data